In [1]:
import sys
sys.path.insert(0, '../../src')
import feedparser
import pandas as pd

import urllib

from postagem.Util import extract_domain, download_and_move_image, get_noticia_comercio
from lexical_analyzer_package import midia_lexical
from midia_postagem import midia_post
from Model.News import News
from Database import midia_table

from newsplease import NewsPlease
from bs4 import BeautifulSoup
import requests

import datetime

In [2]:
link = 'https://www.ocafezinho.com/'

In [3]:
req = requests.get(link)

In [4]:
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='highlight text-center')
noticias = BeautifulSoup(req.text, "html.parser").find_all('div', class_='box')

In [5]:
for noticia in noticias:
    print(noticia.find_all('a', href=True)[0]['href'])
    article = NewsPlease.from_url(noticia.find_all('a', href=True)[0]['href'])
    row = {'titulos': [], 'links': [], 'noticia': [], 'image': [], 'abstract': [], 'date': []}
    if (article is not None):
        row['titulos'].append(article.title)
        row['noticia'].append(article.text)
        row['links'].append(article.url)
        row['abstract'].append(article.text)
        row['date'].append(article.date_publish)
        path_image = article.image_url
        if path_image == '' or path_image == None:
            row['image'].append(0)
        else:
            row['image'].append(download_and_move_image(article.image_url))
        news = News(row['abstract'], row['noticia'], row['date'], row['links'], row['titulos'], row['image'])
        try:
            print(row['titulos'])
            news_in_db = midia_table.check_news(news)
            print('news_in_db: ' + str(news_in_db))
            if (not news_in_db):
                row = pd.DataFrame(row)
                df, categories = midia_lexical.lexical_corpus_and_title(row)
                # DB categories
                if (categories != [set()]):
                    news.set_categories(categories)
                    midia_table.save_news(news)
                    midia_post.post_news(df)
        except:
            print('Empty News')

https://www.ocafezinho.com/2018/09/25/obaoba/
['Ibope: oba-oba cirandeiro não pode contaminar a esquerda']
news_in_db: False
 -- no categories -- 


In [6]:
noticias[0]

<div class="highlight text-center">
<h1><a href="https://www.ocafezinho.com/2018/09/25/obaoba/">Ibope: oba-oba cirandeiro não pode contaminar a esquerda</a></h1>
<a href="https://www.ocafezinho.com/2018/09/25/obaoba/">
<img alt="" class="img-responsive center-block wp-post-image" height="462" sizes="(max-width: 823px) 100vw, 823px" src="https://www.ocafezinho.com/wp-content/uploads/2018/09/bolsonaro1-compressed.jpg" srcset="https://www.ocafezinho.com/wp-content/uploads/2018/09/bolsonaro1-compressed.jpg 823w, https://www.ocafezinho.com/wp-content/uploads/2018/09/bolsonaro1-compressed-400x225.jpg 400w, https://www.ocafezinho.com/wp-content/uploads/2018/09/bolsonaro1-compressed-768x431.jpg 768w" width="823"> </img></a>
</div>